# Imports

In [1]:
# Utils
from datetime import datetime
from datetime import timedelta
import os
import sys
import time
from itertools import combinations

# Data management
import numpy as np
import pandas as pd

# Data fetching
import yfinance as yf

# Spread generation
from sklearn.linear_model import LinearRegression

# Utils

## Stonk price data download

### Input ticker names by industry

In [9]:
def get_tickers_by_industry(industries=None, data_dir=None, filename=None):
    '''
    Read the CSV file containing all tickers and their subindustries and return tickers from the selected subindustries in a list.
    
    -Args:
        industries (List(string)): if not given, return all tickers; else the list can contain:
            'technology_hardware_and_equipment'
            'software_and_services'
            'media_and_entertainment'
            'retailing'
            'automobiles_and_components'
            'semiconductors_and_semiconductor_equipment'
            'health_care_equipment_and_services'
            'banks'
            'pharmaceuticals_biotechnology_and_life_sciences'
            'food_and_staples_retailing'
            'oil_gas_and_consumable_fuels'
            'food_beverage_and_tobacco'
            'telecommunication_services'
            'consumer_durables_and_apparel'
            'consumer_services'
            'transportation'
            'diversified_financials'
            'utilities'
            'capital_goods'
            'insurance'
            'chemicals'
            'metals_and_mining'
            'commercial_and_professional_services'
            'containers_and_packaging'
            'energy_equipment_and_services'
            'construction_materials'
            'paper_and_forest_products'
    
    -Returns:
        tickers (pandas Series): list of selected ticker names
    '''
    filename = 'stonk_list.csv' if filename is None else filename
    data_dir = 'data' if data_dir is None else data_dir
    
    path_to_csv = os.path.join(data_dir, filename)
    stonk_list = pd.read_csv(path_to_csv)
    return stonk_list.set_index('ticker') if industries is None else stonk_list[stonk_list['subindustry'].isin(industries)].set_index('ticker')

In [10]:
def download_stonk_prices(stonk_list, period_years=3, date_from=None, date_to=None, interval='1d', source='yfinance', data_dir='data', proxy=False):    
    '''
    Returns historical price data for the selected stonks.

    -Args:
        stonk_list (List(string)): List of stonk identifiers as strings, case unsensitive
        period_years (float): How many years of data to download until date_to, can be a floating point number
    -Optional:
        date_from (datetime): Start date for stonk data (use instead of period_years)
        date_to (datetime): End date for stonk data
        interval (string): Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        source (string): Where to source data from. Valid sources: yfinance
        data_dir (string): Folder name where to output downloaded data
        file_prefix (string): Prefix of CSV file containing downloaded data inside data_dir
        proxy (boolean): Whether to use a proxy connection to avoid API limits/blocks
                
    -Returns:
        stonks (Pandas Dataframe): Pandas Dataframe containing requested ticker prices
    '''
    
    date_to = datetime.now() if date_to is None else date_to
    date_from = date_to-(timedelta(days=int(365*period_years))) if date_from is None else date_from
    
    if source.lower() == 'yfinance':
        stonks = yf.download(list(stonk_list), start=date_from, end=date_to, interval=interval, group_by='column', threads=True, rounding=True)['Adj Close']
        stonks.dropna(axis=0, how='all', inplace=True)
        stonks.sort_values(by='Date', inplace=True)
        
        stonks.index = pd.to_datetime(stonks.index).date
        stonks.index.name = 'date'

        clean_stonks = stonks.dropna(axis=1, how='all', thresh=len(stonks.index) * 0.95).copy()
        clean_stonks.dropna(axis=0, how='all', thresh=len(clean_stonks.columns) * 0.95, inplace=True)
        clean_stonks.fillna(axis=1, method='ffill', inplace=True)
        clean_stonks.dropna(axis=1, how='any', inplace=True)
        
        # Must be no NA values left
        assert clean_stonks.isna().sum().sum() == 0
    else:
        raise ValueError('Unsupported data source')
        
    def stonks_to_csv(stonks, clean):
        from_date_string = stonks.index[0]
        to_date_string = stonks.index[-1]

        filename = 'stonks_{from_date}_to_{to_date}.csv'.format(from_date=from_date_string, to_date=to_date_string)
        
        if clean:
            filename = 'clean_' + filename
            
        file_path = os.path.join(data_dir, filename)

        stonks.to_csv(path_or_buf=file_path, header=True, index=True, na_rep='NaN')
    
    stonks_to_csv(stonks, clean=False)
    stonks_to_csv(clean_stonks, clean=True)
    
    return (stonks, clean_stonks)

## Stock price data input

In [2]:
def read_stonk_data(date_from, date_to, clean=True, date_index=False, data_dir=None):
    data_dir = 'data' if data_dir is None else data_dir
    data_prefix = 'clean_stonks' if clean else 'stonks'
    
    path = os.path.join(data_dir, '{}_{}_to_{}.csv'.format(data_prefix, date_from, date_to))
    stonks = pd.read_csv(path)
    
    if clean:
        assert stonks.isna().sum().sum() == 0
    
    if date_index:
        return stonks.set_index('date')
    else:
        return stonks.set_index('date').T

In [3]:
def get_stonk_data_by_industry(date_from, date_to, industries=None, stonk_list_filename=None, data_dir=None):
    '''
    Read the CSV file containing all stonk price data and return the tickers from the selected subindustries.
    
    -Args:
        industries (List(string)): if not given, return all tickers; else the list can contain:
            'technology_hardware_and_equipment'
            'software_and_services'
            'media_and_entertainment'
            'retailing'
            'automobiles_and_components'
            'semiconductors_and_semiconductor_equipment'
            'health_care_equipment_and_services'
            'banks'
            'pharmaceuticals_biotechnology_and_life_sciences'
            'food_and_staples_retailing'
            'oil_gas_and_consumable_fuels'
            'food_beverage_and_tobacco'
            'telecommunication_services'
            'consumer_durables_and_apparel'
            'consumer_services'
            'transportation'
            'diversified_financials'
            'utilities'
            'capital_goods'
            'insurance'
            'chemicals'
            'metals_and_mining'
            'commercial_and_professional_services'
            'containers_and_packaging'
            'energy_equipment_and_services'
            'construction_materials'
            'paper_and_forest_products'
    
    -Returns:
        stonks (pandas DataFrame): list of selected tickers' price data
    '''
    all_stonks = read_stonk_data(date_from, date_to, date_index=False, data_dir=data_dir, clean=True)
    
    if industries is None or not industries:
        return all_stonks
    else: 
        all_tickers = get_tickers_by_industry(industries=None, data_dir=data_dir, filename=stonk_list_filename)
        all_stonks = all_stonks.join(all_tickers, how='inner')
        return all_stonks[all_stonks['subindustry'].isin(industries)].drop(columns='subindustry')

In [14]:
# TODO: make combinations with numpy
def combine_stonk_pairs(stonks_prices):
    # All ticker names must be unique
    assert all(stonks_prices.index.unique() == stonks_prices.index)
    assert(len(stonks_prices) < 300)
    
    combs = np.asarray(list(combinations(stonks_prices.index.unique(), 2)))
    
    return stonks_prices.loc[combs[:, 0]], stonks_prices.loc[combs[:, 1]]

## Linear regression residuals

In [5]:
def get_residuals_many(X, Y):
    '''
    Vectorized calculation of residuals from many univariate linear regressions.
        Args:
        - X (numpy array of shape (n_pairs, d_time)): matrix of LR inputs X, each row represents a different regression, corresponding to the same rows in Y
        - Y (numpy array of shape (n_pairs, d_time)): matrix of LR inputs Y, each row represents a different regression, corresponding to the same rows in X
        Returns:
        - residuals (numpy array of shape (n_pairs, d_time)): matrix of resulting residuals between vectorized pairs of X and Y
        - betas (numpy array of shape (n_pairs, 1)): beta coefficients for each linear regression
        - Y_hat (numpy array of shape (n_pairs, d_time)): predictions using X
    '''
    # Stack 2D matrices into 3D matrices
    X = X.reshape(np.shape(X)[0], np.shape(X)[1], -1)
    Y = Y.reshape(np.shape(Y)[0], np.shape(Y)[1], -1)
    
    # Add bias/intercept in the form (Xi, 1)
    Z = np.concatenate([X, np.ones((np.shape(X)[0], np.shape(X)[1], 1))], axis=2)
    
    # Save the transpose as it's used a couple of times
    Z_t = Z.transpose(0, 2, 1)
    
    # Linear Regression equation solutions w.r.t. weight matrix
    # W contains (beta_coef, a_intercept) for each regression
    W = np.matmul(np.linalg.inv(np.matmul(Z_t, Z)),  np.matmul(Z_t, Y))
    
    # Predictions and residuals
    # Y_hat = np.matmul(Z, W).round(2)
    residuals = (Y - np.matmul(Z, W)).round(2)
    
    # Y_hat returned for debugging purposes
    # return (residuals[:, :, 0], W[:, 0, 0], Y_hat[:, :, 0])
    return (residuals[:, :, 0], W[:, 0, 0])

In [85]:
def get_rolling_residuals(X, Y, l_reg, l_roll, dt, data_dir='data'):
    '''
    Calculates rolling window residuals in vectorized form. Returns the result as an array that repeats each ticker for the number of regressions calculated.
    For example, if the inputs are (Pair A, Pair B, Pair C) and l_roll / dt = 3, then the returned results will have the form as follows:
    (Pair A, Pair A, Pair A, Pair B, Pair B, Pair B, Pair C, Pair C, Pair C)
    Works best when l_reg and l_roll are integers.
        Args:
        - X (DataFrame of shape (n_pairs, >= l_reg + l_roll)): matrix of LR inputs X, each row representing not less than complete data period for rolling regressions (can be longer)
        - Y (DataFrame of shape (n_pairs, >= l_reg + l_roll)): matrix of LR inputs Y, each row representing not less than complete data period for rolling regressions (can be longer)
        - l_reg (float): length of each LR to calculate residuals, in years; will be multiplied by the adjusted number of days in a trading year
        - l_roll (float): length of rolling window, in years; will be multipled by the adjusted number of days in a trading year
        - dt (int): rolling window step size, in trading days; total trading year days will be reduced to be divisible by dt (by not more than the value of dt)
        Returns:
        - residuals (numpy array of shape (n_pairs * (l_roll/dt)+1, l_reg + l_roll)): matrix of resulting residuals between vectorized pairs of X and Y
        - betas (numpy array of shape (n_pairs * (l_roll/dt)+1, 1)): beta coefficients for each linear regression
        - Y_hat (numpy array of shape (n_pairs * (l_roll/dt)+1, l_reg + l_roll)): predictions using X
    '''
    
    _DAYS_IN_TRADING_YEAR = 252
    
    # Adjust days in a year so that the number is divisible by dt
    _DAYS_IN_TRADING_YEAR = _DAYS_IN_TRADING_YEAR - (_DAYS_IN_TRADING_YEAR % dt)
    l_reg_days = int(_DAYS_IN_TRADING_YEAR * l_reg)
    l_roll_days = int(_DAYS_IN_TRADING_YEAR * l_roll)
    total_days = l_reg_days + l_roll_days
    n_windows = (l_roll_days // dt) + 1
    n_x = X.shape[0]
    
    X_index = np.repeat(X.index, n_windows)
    Y_index = np.repeat(Y.index, n_windows)
    
    date_index = X.columns[-total_days:]
    date_index_windowed = np.empty(shape=(n_x*n_windows, 2), dtype='O')
    
    X = X.to_numpy().astype(np.float64)
    Y = Y.to_numpy().astype(np.float64)
    
    # Rolling window length must be divisible by dt
    assert (l_roll_days % dt) == 0
    
    # There has to be enough days' worth of data in X (and Y) and their shapes must match
    assert X.shape == Y.shape and X.shape[1] >= total_days
    
    # Take the total_days from the end of the arrays (most recent days first, oldest days at the end are cut off)
    X = X[:, -total_days:]
    Y = Y[:, -total_days:]
    
    # Create empty arrays that will contain windowed slices of our data
    X_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    Y_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    
    # Take windowed slices and place them into the created empty arrays
    for n in range(n_x):
        for i in range(n_windows):
            X_windows[(n*n_windows)+i] = X[n, i*dt:l_reg_days+(i*dt)]
            Y_windows[(n*n_windows)+i] = Y[n, i*dt:l_reg_days+(i*dt)]
            date_index_windowed[(n*n_windows)+i, 0] = date_index[i*dt]
            date_index_windowed[(n*n_windows)+i, 1] = date_index[l_reg_days+(i*dt)-1]
    
    # Make sure we've got the windowing dimensions right
    assert X_windows.shape == (n_x*n_windows, l_reg_days) and Y_windows.shape == (n_x*n_windows, l_reg_days)
    
    # Sanity check
    assert all([
        X[0, -1] == X_windows[n_windows-1, -1],
        Y[0, -1] == Y_windows[n_windows-1, -1],
        X[-1, -1] == X_windows[-1, -1],
        Y[-1, -1] == Y_windows[-1, -1],
    ])
    
    # Construct index column
    XY_index = pd.DataFrame(np.array([X_index, Y_index, date_index_windowed[:, 0], date_index_windowed[:, 1]])).apply('_'.join, axis=0, raw=True)
    
    # Calculate and return the residuals
    res, betas = get_residuals_many(X_windows, Y_windows)
    
    res = pd.DataFrame(res, index=XY_index)
    betas = pd.DataFrame(betas, index=XY_index)
    
    time = datetime.now().time()
    res.to_csv(os.path.join(data_dir, time.strftime('residuals_%H%M%S.csv')), header=False, index=True)
    betas.to_csv(os.path.join(data_dir, time.strftime('betas_%H%M%S.csv')), header=False, index=True)
    pd.Series(date_index).to_csv(os.path.join(data_dir, time.strftime('dates_%H%M%S.csv')), header=False, index=False)
    
    return res, betas

## Pipeline example tutorial

#### 1. Download stock daily prices

In [106]:
# Gets all ticker names (no argument given)
ticker_list = get_tickers_by_industry()

In [ ]:
ticker_list

In [ ]:
# Specific date - 3rd of March 2022 (Y, M, D)
date_to = datetime(2022, 3, 1)
# Date of today
date_to = datetime.today()
# How many years' of data to download (going backwards from date_end). Year can be a floating point number
period_years = 4

In [108]:
# Download ticker price data for the tickers selected above (saved to .csv automatically)
df, df_clean = download_stonk_prices(ticker_list.index, period_years=period_years, date_to=date_to)

[*********************100%***********************]  2283 of 2283 completed

9 Failed downloads:
- WFC PRN: No data found, symbol may be delisted
- ET-PD: No data found for this date range, symbol may be delisted
- SNX.VI: No data found, symbol may be delisted
- ET-PE: No data found for this date range, symbol may be delisted
- FHN PRA: No data found, symbol may be delisted
- FTAI-PA: No data found for this date range, symbol may be delisted
- ET-PC: No data found for this date range, symbol may be delisted
- ALL-PB: No data found for this date range, symbol may be delisted
- RXN.VI: No data found, symbol may be delisted


In [159]:
df_clean

,A,AA,AAL,AAON,AAP,AAPL,AAWW,AAXJ,AB,ABBV,...,ZGNX,ZION,ZIONO,ZIONP,ZNGA,ZS,ZTS,ZUMZ,ZUO,ZWS
date,,,,,,,,,,,,,,,,,,,,,
2018-03-08,67.70,47.37,53.50,35.45,112.53,42.61,60.55,73.75,19.00,94.68,...,44.50,51.12,21.49,24.43,3.71,3.71,80.63,20.25,20.25,29.87
2018-03-09,68.94,47.76,54.75,36.91,115.56,43.34,61.55,75.37,19.10,96.52,...,45.00,51.98,21.66,24.43,3.72,3.72,82.56,19.55,19.55,30.68
2018-03-12,68.54,48.25,55.17,37.25,115.12,43.76,61.30,75.60,19.00,95.72,...,44.80,51.56,21.54,24.41,3.75,3.75,82.45,19.90,19.90,30.12
2018-03-13,68.51,48.48,54.91,37.69,115.69,43.33,62.35,75.06,18.75,96.90,...,44.50,50.80,21.64,24.39,3.70,3.70,82.63,20.00,20.00,30.00
2018-03-14,67.77,46.47,53.98,37.55,113.81,42.97,62.85,75.33,18.47,96.24,...,44.50,49.92,21.62,24.05,3.77,3.77,82.39,19.60,19.60,29.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28,130.36,75.34,17.25,58.56,204.48,165.12,78.36,78.62,44.84,147.77,...,26.26,70.89,26.16,23.54,9.08,239.15,193.65,44.49,15.18,32.52
2022-03-01,131.93,79.78,16.29,57.18,201.17,163.20,80.39,77.91,43.38,147.69,...,26.36,64.94,26.00,22.17,9.00,247.83,192.54,43.04,15.21,31.42
2022-03-02,133.16,82.56,16.37,55.42,209.07,166.56,80.84,78.03,44.81,149.57,...,26.37,68.38,25.74,22.70,9.12,254.41,196.07,45.00,15.40,33.40


#### 2. Read stock data by industry

In [31]:
# Get stock data from selected industries only
stonks = get_stonk_data_by_industry('2018-03-08', '2022-03-04', industries=['consumer_durables_and_apparel', 'consumer_services'])

In [27]:
# Get stock data of ALL industries (all tickers) - no arguments specified
stonks = get_stonk_data_by_industry('2018-03-08', '2022-03-04')

#### 3. Input list of ticker pairs from CSV and select price data

In [ ]:
ticker_pairs = 

#### 4. Calculate residuals (exported to CSV)

In [151]:
t1 = time.time()
residuals, betas = get_rolling_residuals(X, Y, l_reg=3, l_roll=1, dt=5)
t2 = time.time()
print("Time: " + str(int(t2-t1)) + 's') 

Time: 4s


## Other

### Slow residual functions (for testing)

In [154]:
# def get_rolling_slow_residuals(X, Y, l_reg, l_roll, dt):
#     _DAYS_IN_TRADING_YEAR = (252) - (252 % dt)
#     l_reg_days = _DAYS_IN_TRADING_YEAR * l_reg
#     l_roll_days = _DAYS_IN_TRADING_YEAR * l_roll
#     total_days = l_reg_days + l_roll_days
#     n_windows = l_roll_days // dt
#     n_x = X.shape[0]
    
#     assert (l_roll_days % dt) == 0
#     assert X.shape[1] >= total_days and Y.shape[1] >= total_days
    
#     X = X[:, -total_days:]
#     Y = Y[:, -total_days:]
    
#     # First window
#     X_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
#     Y_windows = np.empty(shape=(n_x*n_windows, l_reg_days))
    
#     for n in range(n_x):
#         for i in range(n_windows):
#             X_windows = np.concatenate(( X_windows, X[n, i*dt:l_reg_days+(i*dt)] ))
#             Y_windows = np.concatenate(( Y_windows, Y[n, i*dt:l_reg_days+(i*dt)] ))
    
#     assert X_windows.shape == (n_x*n_windows, l_reg_days) and Y_windows.shape == (n_x*n_windows, l_reg_days)
    
#     return get_slow_residuals_many(X_windows, Y_windows)

In [104]:
# def get_slow_residuals_many(X, Y, n_jobs=-1):
#     lr = LinearRegression(n_jobs=n_jobs, fit_intercept=True)
#     X = X.reshape((X.shape[0], X.shape[1], -1))
#     Y = Y.reshape((Y.shape[0], Y.shape[1], -1))
    
#     preds = []
#     res = []
#     betas = []
#     for i in range(X.shape[0]):
#         lr.fit(X[i], Y[i])
#         preds.append(lr.predict(X[i]).round(2))
#         res.append(Y[i]-preds[-1])
#         betas.append(lr.coef_[0][0])
#     return (np.asarray(res)[:,:,0], np.asarray(preds)[:,:,0], np.asarray(betas))

In [28]:
# t1_fast = time.time()
# res, betas, preds  = get_rolling_residuals(X, Y, l_reg=2, l_roll=1, dt=5)
# t2_fast = time.time()

# t1_slow = time.time()
# res_slow, preds_slow = get_rolling_slow_residuals(X, Y, l_reg=2, l_roll=1, dt=5)
# t2_slow = time.time()

# print("Time slow: " + str(t2_slow-t1_slow))
# print("Time fast: " + str(t2_fast-t1_fast))

In [29]:
# t1_fast = time.time()
# res, preds, betas = get_residuals_many(X, Y)
# t2_fast = time.time()

# t1_slow = time.time()
# res_slow, preds_slow, betas_slow = get_slow_residuals_many(X, Y)
# t2_slow = time.time()

# print("Time slow: " + str(t2_slow-t1_slow))
# print("Time fast: " + str(t2_fast-t1_fast))

### Stock list preprocessing

In [153]:
def preprocess_stock_list(raw_data_path='data/raw_stonk_list.xls', output_path='data/stonk_list.csv'):
    '''
    Parses a raw excel file from CapitalIQ containing ticker names and their subindustries, validates
    unusual ticker names with Yahoo Finance, saving the processed data in CSV format.

        Parameters:
            Required:
                raw_data_path (string):
                    Path to the raw excel file.
                output_path (string):
                    Path where to save the parsed data.
                
        Returns:
            Nothing
    '''
    
    df = pd.read_excel(io=raw_data_path)
    
    # Drop NA rows
    df.dropna(axis=0, inplace=True)
    
    # Reset index and drop the first row
    df.reset_index(inplace=True, drop=True)
    df.drop(index=0, axis=0, inplace=True)
    
    # Drop unwanted columns
    df.drop(columns=df.columns[[1, 2, 3, 4, 5, 7, 8, 9]], inplace=True)
    
    # Rename remaining columns
    df.columns = ['ticker', 'subindustry']
    
    # Remove the '(Primary)' tag from subindustries
    df['subindustry'] = df['subindustry'].str.replace(r' \(Primary\)', '')
    
    # Remove everything until (and including) the semicolon for tickers
    df['ticker'] = df['ticker'].str.replace(r'(.*:)', '')
    
    df['ticker'] = df['ticker'].str.replace(r' WI', '.VI')
    df['ticker'] = df['ticker'].str.replace(r'\.WI', '.VI')
    
    # Replace the ticker endings for a Yahoo finance supported format
    df['ticker'] = df['ticker'].str.replace(r'\.PR', '-P')
    # df['ticker'] = df['ticker'].str.replace(r' PR', '-P')
    
    # Take all remaining tickers that have a dot
    dotted = df[df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Replace the dots with dashes
    dashed = dotted.copy()
    dashed['ticker'] = dashed['ticker'].str.replace(r'\.', '-')
    
    # Remove the dots
    undotted = dotted.copy()
    undotted['ticker'] = undotted['ticker'].str.replace(r'\.', '')

    # Combine all variantas together
    all_variants = pd.concat([dotted, dashed, undotted])
    
    # Run all of these through Yahoo finance, get last day's price
    stonks = yf.download(list(all_variants['ticker'].astype('string').values), period='1m', interval='1d', group_by='column')
    
    # Drop all NA tickers (that failed to download)
    valid_tickers = stonks['Adj Close'].iloc[-1].dropna(axis=0).to_frame().reset_index()
    
    # Rename columns
    valid_tickers.columns = ['ticker', 'price']
    
    # Add subindustries to the remaining valid tickers
    valid_tickers = valid_tickers.join(all_variants.set_index('ticker'), on='ticker')
    
    # Drop the price column
    valid_tickers.drop(columns=valid_tickers.columns[[1]], inplace=True)
    
    # Remove all tickers that have a dot from main dataframe
    df = df[~df['ticker'].str.fullmatch(r'[A-Z]*\.[A-Z]')]
    
    # Add the validated tickers back
    df = pd.concat([df, valid_tickers], axis=0, ignore_index=True)
    
    # Make the subindustry strings more code friendly
    df['subindustry'] = df['subindustry'].str.replace(' ', '_')
    df['subindustry'] = df['subindustry'].str.lower()
    df['subindustry'] = df['subindustry'].str.replace(',', '')
    
    df.to_csv(path_or_buf=output_path, header=True, index=False)